In [1]:
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model, Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet201
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras.backend as K
%matplotlib inline

In [2]:
X_train = np.load("128_128_train.npy")
y_train = np.load("train_labels.npy")
X_val = np.load("128_128_val.npy")
y_val = np.load("val_labels.npy")
X_test = np.load("128_128_test.npy")
y_test = np.load("test_labels.npy")

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
input_shape = X_val[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [5]:
pre_trained_model = InceptionV3(input_shape=(128, 128, 3), include_top=False, weights="imagenet")

In [6]:
for layer in pre_trained_model.layers:
    print(layer.name)
    if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
        layer.trainable = True
        K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
        K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
    else:
        layer.trainable = False

print(len(pre_trained_model.layers))

input_2
conv2d
batch_normalization
activation
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
max_pooling2d
conv2d_3
batch_normalization_3
activation_3
conv2d_4
batch_normalization_4
activation_4
max_pooling2d_1
conv2d_8
batch_normalization_8
activation_8
conv2d_6
conv2d_9
batch_normalization_6
batch_normalization_9
activation_6
activation_9
average_pooling2d
conv2d_5
conv2d_7
conv2d_10
conv2d_11
batch_normalization_5
batch_normalization_7
batch_normalization_10
batch_normalization_11
activation_5
activation_7
activation_10
activation_11
mixed0
conv2d_15
batch_normalization_15
activation_15
conv2d_13
conv2d_16
batch_normalization_13
batch_normalization_16
activation_13
activation_16
average_pooling2d_1
conv2d_12
conv2d_14
conv2d_17
conv2d_18
batch_normalization_12
batch_normalization_14
batch_normalization_17
batch_normalization_18
activation_12
activation_14
activation_17
activation_18
mixed1
conv2d_22
batch_normalization_22
activation_22
conv2d

In [7]:
last_layer = pre_trained_model.get_layer('mixed10')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 2, 2, 2048)


In [8]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

c:\Users\muham\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [10]:
batch_size = 64
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

<ipython-input-10-86fe2f61745c>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/3
126/126 [==============================] - 298s 2s/step - loss: 1.3214 - accuracy: 0.6299 - val_loss: 5.8228 - val_accuracy: 0.6540
Epoch 2/3
126/126 [==============================] - 276s 2s/step - loss: 1.0982 - accuracy: 0.6676 - val_loss: 1.0574 - val_accuracy: 0.6786
Epoch 3/3
126/126 [==============================] - 277s 2s/step - loss: 1.0600 - accuracy: 0.6694 - val_loss: 1.0200 - val_accuracy: 0.6629


In [11]:
for layer in pre_trained_model.layers[249:]:
    layer.trainable = True

In [12]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [13]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [14]:
batch_size = 64
epochs = 20
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

<ipython-input-14-2b1cd3556215>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/20
126/126 [==============================] - 285s 2s/step - loss: 0.9451 - acc: 0.6853 - val_loss: 0.9194 - val_acc: 0.6808 - lr: 1.0000e-04
Epoch 2/20
126/126 [==============================] - 282s 2s/step - loss: 0.8533 - acc: 0.7036 - val_loss: 0.8240 - val_acc: 0.7054 - lr: 1.0000e-04
Epoch 3/20
126/126 [==============================] - 280s 2s/step - loss: 0.8026 - acc: 0.7240 - val_loss: 0.8414 - val_acc: 0.6942 - lr: 1.0000e-04
Epoch 4/20
126/126 [==============================] - 282s 2s/step - loss: 0.7645 - acc: 0.7270 - val_loss: 0.8344 - val_acc: 0.7098 - lr: 1.0000e-04
Epoch 5/20
126/126 [==============================] - 280s 2s/step - loss: 0.7480 - acc: 0.7370 - val_loss: 0.8486 - val_acc: 0.7076 - lr: 1.0000e-04
Epoch 6/20
126/126 [==============================] - 284s 2s/step - loss: 0.7054 - acc: 0.7489 - val_loss: 0.7807 - val_acc: 0.7143 - lr: 1.0000e-04
Epoch 7/20
126/126 [==============================] - 280s 2s/step - loss: 0.6989 - acc: 0.7490 - va

In [15]:
model.save("model_inceptionV3.h5")

In [16]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

29/29 [==============================] - 9s 297ms/step - loss: 0.6938 - acc: 0.7572
Validation: accuracy = 0.757206  ;  loss_v = 0.693757
Test: accuracy = 0.750499  ;  loss = 0.708717
